# Исследование надежности заемщиков.

**Цель исследования** — проверить гипотезу о влиянии семейного положения и количества детей клиента на факт погашения кредита в срок.

## Обзор данных

Выведем на экран первые 10 строк таблицы.

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv(r'C:\Users\ANN\Downloads/dataset.csv')    
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


С помощью метода info() получили информацию о названиях колонок, количестве строк. </b></font><br>В таблице 12 столбцов, тип данных - int, float и object. </b></font><br>Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.</b></font><br>В столбце days_employed есть значения со значением минус, возможно, эта аномалия связана с некорректной выгрузкой данных.
</b></font><br></b></font><br> Согласно документации к данным: </b></font><br>

* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

### Заполнение пропусков

In [4]:
df.isna().sum()    

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

С мощью методов isna() и sum() обнаружили, что в колонках total_income - пропущенных значений - 2174 ;</b></font><br>
в колонке days_employed - пропущенных значений - 2174.</b></font><br> 
Предположим, что пропуски в данных колонках связаны с намеренным нежеланием заемщиков предоставлять эту информацию.


Для определения долей пропусков можем использовать следующий способ:

In [5]:
for col in df.columns:
    pct_missing = np.mean(df[col].isna())
    print('{} - {}%'.format(col, round(pct_missing*100)))

children - 0%
days_employed - 10%
dob_years - 0%
education - 0%
education_id - 0%
family_status - 0%
family_status_id - 0%
gender - 0%
income_type - 0%
debt - 0%
total_income - 10%
purpose - 0%


Пропущенные значения в столбце days_employed составлют 10%, в total_income - 10%.

Заполним пропуски в столбце total_income медианным значением, которое будет "серединным" между наибольшим и наименьшим значениями. Строки с пропусками не удаляем, потому что они составляют существенную часть от общего числа. Проверим результат.

In [6]:
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform("median"))
df['total_income']

0        253875.639453
1        112080.014102
2        145885.952297
3        267628.550329
4        158616.077870
             ...      
21520    224791.862382
21521    155999.806512
21522     89672.561153
21523    244093.050500
21524     82047.418899
Name: total_income, Length: 21525, dtype: float64

### Проверка данных на аномалии и исправления.

Проверим есть ли в столбце children отрицательные значения, если да, то изменим на положительные. Проверим результат.

In [7]:
df['children'].sort_values().unique()

array([-1,  0,  1,  2,  3,  4,  5, 20], dtype=int64)

Поменяем отрицательное значение на положительное:

In [8]:
df['children'] = df['children'].abs()

В столбце есть аномалия - клиенты с 20ю детьми. Предположим, что данные были заполнены некорректно, правильное занчение должно быть 2. Изменим значение и проверим результат

In [9]:
df.loc[df['children'] == 20, 'children'] = 2
df['children'].sort_values().unique()

array([0, 1, 2, 3, 4, 5], dtype=int64)

С помощью конструкции try...except... проверим, есть ли пропуски в столбце days_employed. Если пропуски есть, результатом проверки будет - ошибка.

In [10]:
try:
    df[df['days_employed'].isna].index
except:
    print('ошибка')

ошибка


В столбце days_employed есть значения со знаком минус, изменим эти значения на положительные.

In [11]:
df['days_employed'] = df['days_employed'].abs()
df['days_employed']

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

Заполним пропуски в столбце total_income медианным значением, которое будет "серединным" между наибольшим и наименьшим значениями. Проверим результат.

In [12]:
df_median = df['days_employed'].median()
df['days_employed'].fillna(value=df_median)

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

### Изменение типов данных.

Изменим вещественный тип данных в столбце total_income на целочисленный c помощью метода astype(), потому что в дальнейшем нам будет проще совершать вычисления с целым числом. Проверим результат.

In [13]:
df['total_income'] = df['total_income'].astype('int')
df['total_income']

0        253875
1        112080
2        145885
3        267628
4        158616
          ...  
21520    224791
21521    155999
21522     89672
21523    244093
21524     82047
Name: total_income, Length: 21525, dtype: int32

### Удаление дубликатов.

Посчитаем количество явных дубликатов с помощью методов duplicated() и sum(), удалим явные дубликаты и проверим результат.

In [14]:
df.duplicated().sum()

54

In [15]:
df = df.drop_duplicates().reset_index(drop=True)

In [16]:
df.duplicated().sum()

0

В столбце education education можно встретить написание с заглавной буквы, строчные символы, крупный шрифт. Сделаем все символы в толбце строчными с помощью метода str.lower() и проверим результат. 

In [17]:
df['education'].value_counts()
df['education']

0         высшее
1        среднее
2        Среднее
3        среднее
4        среднее
          ...   
21466    среднее
21467    среднее
21468    среднее
21469    среднее
21470    среднее
Name: education, Length: 21471, dtype: object

In [18]:
df['education'] = df['education'].str.lower()
df['education'].head(5)

0     высшее
1    среднее
2    среднее
3    среднее
4    среднее
Name: education, dtype: object

В столбце purpose имеются неявные дубликаты. Причина их появления - человеческий фактор: заемщики заполняли данные вручную, формулировки целей могут отличатся. Выведем на экран список уникальных значений в столбце.

In [19]:
df['purpose'].sort_values().unique()

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

Чтобы очистить от неявных дубликатов столбец, напишем функцию `replace_wrong_purposes()`.

In [20]:
def replace_wrong_purposes(purpose):            
    if  'авто' in purpose:
        return 'операции с автомобилем'
    elif 'недвиж' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образо' in purpose:
        return 'получение образования'
    
df['purpose']=df['purpose'].apply(replace_wrong_purposes)

Выведем на экран уникальные значения столбца purpose.

In [21]:
df['purpose'].sort_values().unique()

array(['операции с автомобилем', 'операции с недвижимостью',
       'получение образования', 'проведение свадьбы'], dtype=object)

**Выводы**

Предобработка обнаружила следующие проблемы в данных:

- пропущенные значения,
- аномалии в данных,
- некорретные типы данных,
- дубликаты — явные и неявные.

Пропущенные значения заменили на медианные значения, устранили аномалии, изменили тип данных, удалили явные дубликаты и сократили количество неявных дубликатов.


### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим словари для оптимизации данных, в которых:
каждому уникальному значению из столбца education будет соответствовать уникальное значение education_id;
каждому уникальному значению из столбца df_family_status - family_status_id.

In [22]:
df_education = df[['education_id','education']]
df_education.head(10)

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее
5,0,высшее
6,0,высшее
7,1,среднее
8,0,высшее
9,1,среднее


In [23]:
df_family_status = df[['family_status_id','family_status']]
df_family_status.head(10)

,family_status_id,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,1,гражданский брак
5,1,гражданский брак
6,0,женат / замужем
7,0,женат / замужем
8,1,гражданский брак
9,0,женат / замужем


Удалим из датафрейма столбцы education и family_status.

In [24]:
df = df.drop(columns=['education','family_status'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21471 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21471 non-null  int64  
 3   education_id      21471 non-null  int64  
 4   family_status_id  21471 non-null  int64  
 5   gender            21471 non-null  object 
 6   income_type       21471 non-null  object 
 7   debt              21471 non-null  int64  
 8   total_income      21471 non-null  int32  
 9   purpose           21471 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(3)
memory usage: 1.6+ MB


### Категоризация дохода.

Чтобы отфильтровать столбец total_income, создадим  функцию total_income_category. Деление данных на категории упростит работу с таблицей, обращаться можно будет не к значению, а к названию категории.

In [25]:
def total_income_category(total_income):
    if total_income <= 30000:
        return 'E'
    if total_income <= 50000:
        return 'D'
    if total_income <= 200000:
        return 'C'
    if total_income <= 1000000:
        return 'B'
    return 'A'



df['total_income_category']=df['total_income'].apply(total_income_category)
df['total_income_category'].value_counts()

C    16032
B     5042
D      350
A       25
E       22
Name: total_income_category, dtype: int64

### Категоризация целей кредита.

Создадим столбец purpose_category с категориями целей кредита обращаться можно будет не к значению, а к названию категории.

In [26]:
def purpose_category(purpose):
    if 'автомоб' in  purpose:
        return 'операции с автомобилем'
    if 'образов' in  purpose:
        return 'получение образования'
    if 'недвиж' in purpose or 'жилье' in  purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in  purpose:
        return 'проведение свадьбы'
    if 'ремонт'in  purpose:
        return 'ремонт жилья'
  
df['purpose_category']=df['purpose'].apply(purpose_category)
df['purpose_category'].value_counts()

операции с недвижимостью    10814
операции с автомобилем       4308
получение образования        4014
проведение свадьбы           2335
Name: purpose_category, dtype: int64

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?


Среди заемщиков с количеством детей - четыре, существует вероятность не выплаты кредита в срок.
Среди заемщиков с пятью детьми должников нет.

In [27]:
df.pivot_table(index='children', values='debt', aggfunc=['mean', 'count'])

,mean,count
,debt,debt
children,,
0,0.075353,14107
1,0.091639,4856
2,0.094925,2128
3,0.081818,330
4,0.097561,41
5,0.000000,9


Среди заемщиков, статус которых - в гражданском браке и не женат/не замужем существует вероятность не выплаты кредита в срок.

In [28]:
df.pivot_table(index='family_status_id', values='debt', aggfunc=['mean', 'count'])

,mean,count
,debt,debt
family_status_id,,
0,0.075421,12344
1,0.093202,4163
2,0.065693,959
3,0.071130,1195
4,0.097509,2810


Среди заемщиков, ежемесячный доход, которых не превышает 30 000, существует вероятность не выплаты кредита в срок.


In [29]:
df.pivot_table(index='total_income_category', values='debt', aggfunc=['mean', 'count'])

,mean,count
,debt,debt
total_income_category,,
A,0.080000,25
B,0.070607,5042
C,0.084830,16032
D,0.060000,350
E,0.090909,22


Среди заемщиков, цель получения кредита которых - операции с автомобилем и получение образования, существует вероятность не выплаты кредита в срок.

In [30]:
df.pivot_table(index='purpose_category', values='debt', aggfunc=['mean', 'count'])

,mean,count
,debt,debt
purpose_category,,
операции с автомобилем,0.093547,4308
операции с недвижимостью,0.072314,10814
получение образования,0.092177,4014
проведение свадьбы,0.079657,2335


##### Вывод 1:

Гипотеза,о том что семейное положение и количество детей клиента банка частично подтвердилась:

cреди заемщиков с количеством детей - четыре, существует вероятность не выплаты кредита в срок.
среди заемщиков, статус которых - в гражданском браке и не женат/не замужем существует вероятность не выплаты кредита в срок. 

## Общий вывод:

Гипотеза,о том что семейное положение и количество детей клиента банка частично подтвердилась:

cреди заемщиков с количеством детей (41 клиент из общего количества - 21 525 человек) четыре, существует вероятность не выплаты кредита в срок, но среди заемщиков с пятью детьми должников нет (клиентов с пятью детьми 9 из 21 525 человек);

среди заемщиков, статус которых - в гражданском браке и не женат/не замужем(4 163 и 2810 клиентов), существует вероятность не выплаты кредита в срок, среди заемщиков статус которых(959 клиентов) - вдовец/вдова эта вероятность минимальна.

Также мы проверили влияет ли ежемесячный доход и цель получения кредита на погашение кредиты в срок:

 среди заемщиков, ежемесячный доход которых составляет от 0 до 30 тыс.(22 клиента) существует вероятность не выплаты кредита в срок, среди заемщиков, ежемесячный доход которых составляет от 31 тыс. до 50 тыс.(350 клиентов), эта вероятность минимальна;

 среди заемщиков, цель получения кредита которых - получение образования (4014 клиентов) и операции с автомобилем (4308 клиентов) существует вероятность не выплаты кредита в срок.
